In [1]:
#libraries
import sys
import numpy as np
import matplotlib.pyplot as plt
import pickle
import os
import gzip
import model
from torch.utils import data
import torch
import torch.optim as optim
from torch.autograd import Variable
from time import gmtime, strftime
import sys
import torch.nn as nn

ModuleNotFoundError: No module named 'torch'

In [ ]:
#variables
use_gpu = 0
conv2d1_filters_numbers = 8
conv2d1_filters_size = 9
conv2d2_filters_numbers = 8
conv2d2_filters_size = 1
conv2d3_filters_numbers = 1
conv2d3_filters_size = 5
down_sample_ratio = 16
epochs = 50
HiC_max_value = 100 # ?????
batch_size = 256


In [ ]:
#loading data
low_resolution_samples = np.load("../data/divided-data/GM12878_primary_down_chr1-17.npy").astype(np.float32) * down_sample_ratio
low_resolution_samples = np.expand_dims(low_resolution_samples, axis=1)
high_resolution_samples = np.load("../data/divided-data/GM12878_primary_chr1-17.npy").astype(np.float32)
high_resolution_samples = np.expand_dims(high_resolution_samples, axis=1)

In [ ]:
#changing high resolution data size
sample_size = low_resolution_samples.shape[-1]
padding = conv2d1_filters_size + conv2d2_filters_size + conv2d3_filters_size - 3
half_padding = padding / 2
high_resolution_samples = high_resolution_samples[:,:,half_padding:(sample_size-half_padding),half_padding:(sample_size-half_padding)]

In [ ]:
#loading data to tensor
# still not getting why np.zeros has been added
lowres_set = data.TensorDataset(torch.from_numpy(low_resolution_samples), torch.from_numpy(np.zeros(low_resolution_samples.shape[0])))
lowres_loader = torch.utils.data.DataLoader(lowres_set, batch_size=batch_size, shuffle=False)

hires_set = data.TensorDataset(torch.from_numpy(high_resolution_samples), torch.from_numpy(np.zeros(high_resolution_samples.shape[0])))
hires_loader = torch.utils.data.DataLoader(hires_set, batch_size=batch_size, shuffle=False)

In [ ]:
#defining model
Net = model.Net(40, 28)
if use_gpu:
    Net = Net.cuda()

optimizer = optim.SGD(Net.parameters(), lr = 0.00001)
_loss = nn.MSELoss()
Net.train()
running_loss = 0.0
losslist = []
log = open('HindIII_train.txt', 'w')

In [ ]:
#training model
for epoch in range(0, epochs):
    # iterate over two lists and their indices using enumerate together with zip
    # lowres_loader is list of batches
    for i, (v1, v2) in enumerate(zip(lowres_loader, hires_loader)):
        # probably it is for skipping last incomplete batch
        if (i == len(lowres_loader) - 1):
            continue


        # v1 is list with length = 2. v1[0] is data tensor so with shape 256*1*40*40. v1[1] is vector of 256 zeros because pf line 85 but what's the reason?
        _lowRes, _ = v1
        _highRes, _ = v2
        # print "_lowres:", _lowRes, "\n shape: ", _lowRes.shape

        _lowRes = Variable(_lowRes)
        _highRes = Variable(_highRes)


        if use_gpu:
            _lowRes = _lowRes.cuda()
            _highRes = _highRes.cuda()
        optimizer.zero_grad()
        y_prediction = Net(_lowRes)
        loss = _loss(y_prediction, _highRes)
        loss.backward()
        optimizer.step()
        #print(loss.item())
        running_loss += loss.item()

    print '-------', i, epoch, running_loss/i, strftime("%Y-%m-%d %H:%M:%S", gmtime())
    losslist.append(running_loss/i)
    log.write(str(epoch) + ', ' + str(running_loss/i,) + '\n')
    running_loss = 0.0
    # save the model every 10 epoches
    if (epoch % 10 == 0):
        if not os.path.exists('../models/19-06-05/'):
            os.makedirs('../models/19-06-05/')
        torch.save(Net.state_dict(), '../models/19-06-05/' + str(epoch))
        pass
    



In [ ]:
plt.plot(test_cor_list, label = "test_cor")
plt.plot(train_cor_list, label = "train_cor")
#plt.axis([0, 6, 0, 20])
plt.show()

In [ ]:
test_y_prediction = Net(torch.from_numpy(test_low_resolution_samples)).detach().numpy()
    train_y_prediction = Net(torch.from_numpy(train_low_resolution_samples)).detach().numpy()
    ### evaluating part ###
    test_cor = np.nanmean([corr_highVSlow(i,test_y_prediction,test_high_resolution_samples) for i in range(test_low_resolution_samples.shape[0])])
    train_cor = np.nanmean([corr_highVSlow(i,train_y_prediction,train_high_resolution_samples) for i in range(train_low_resolution_samples.shape[0])])
    print("test data correlation: ", test_cor)
    print("train data correlation: ", train_cor)
    test_cor_list.append(test_cor)
    train_cor_list.append(train_cor)